In [1]:
import pandas as pd
import numpy as np
import microdf as mdf

p = pd.read_csv("../data/asec_2019_ipums.csv.gz")

# Preprocess
p.columns = p.columns.str.lower()

# Replace NIU codes.
NIU_CODES = {
    "adjginc": 99999999,
    "offtotval": 9999999999,
    # Secondary individuals under 15 have NIU for cutoff.
    # We set them to zero as to give them no UBI
    # (they're still part of SPM units with others who get UBI).
    "cutoff": 999999,
    "incwelfr": 999999,
}
for column, niu in NIU_CODES.items():
    p[column].replace({niu: 0}, inplace=True)

p["fam"] = p.marbasecidh.astype(str) + "-" + p.famid.astype(str)

# Create aggregates.

# Family: note families are assigned only to one SPM unit.
f = p.groupby(["fam", "cutoff", "offtotval", "spmfamunit"]).adjginc.sum().reset_index().rename(columns={"adjginc": "fam_adjginc"})

# SPM unit, with all SPM characteristics.
s = p.groupby(['spmlunch', 'spmcaphous', 'spmwt', 'spmeitc',
       'spmwic', 'spmheat', 'spmsnap', 'spmtotres', 'spmthresh', 'spmfamunit']).incwelfr.sum().reset_index().rename(columns={"incwelfr": "spmincwelfr"})

In [2]:
def phase_out(amount, rate, respect_to):
    return np.maximum(0, amount - respect_to * rate)


def ubi_fpg(phase_out_rate=0, respect_to="offtotval", repeal_bens=None):
    f["chg"] = phase_out(f.cutoff, phase_out_rate, np.maximum(0, f[respect_to]))
    s2 = s.merge(f.groupby("spmfamunit").chg.sum().reset_index(), on="spmfamunit")
    if repeal_bens is not None and repeal_bens != [""]:
        s2.chg -= s2[repeal_bens].sum(axis=1)
    p2 = p.merge(s2[["chg", "spmfamunit"]], on="spmfamunit")
    p2.spmtotres += p2.chg
    p2["loser"] = p2.chg < -1
    pov = mdf.poverty_rate(p2, "spmtotres", "spmthresh", "asecwt")
    cost = mdf.weighted_sum(s2, "chg", "spmwt")
    pct_loser = mdf.weighted_mean(p2, "loser", "asecwt")
    return pd.Series({"poverty": pov, "cost": cost, "pct_loser": pct_loser})

In [3]:
sim = mdf.cartesian_product({
    "phase_out": [np.inf, 0.5, 0.25, 0],
    "repeal_bens": ["", "spmeitc,spmcaphous,spmsnap,spmincwelfr,spmlunch"]
    })
sim = pd.concat([
    sim,
    sim.apply(lambda x: ubi_fpg(x.phase_out, repeal_bens=x.repeal_bens.split(",")), axis=1)
    ], axis=1)
cur_pov = sim[(sim.phase_out == np.inf) & (sim.repeal_bens == "")].poverty.values[0]

def pct_chg(base, new):
    return (new - base) / base

sim["pov_pov_chg"] = pct_chg(cur_pov, sim.poverty)
sim

,phase_out,repeal_bens,poverty,cost,pct_loser,pov_pov_chg
0,inf,,0.117132,0.000000e+00,0.000000,0.000000
1,inf,"spmeitc,spmcaphous,spmsnap,spmincwelfr,spmlunch",0.150437,-1.075703e+11,0.399615,0.284336
2,0.50,,0.031919,3.442652e+11,0.000000,-0.727499
3,0.50,"spmeitc,spmcaphous,spmsnap,spmincwelfr,spmlunch",0.047664,2.366949e+11,0.235764,-0.593073
4,0.25,,0.018187,7.552311e+11,0.000000,-0.844727
5,0.25,"spmeitc,spmcaphous,spmsnap,spmincwelfr,spmlunch",0.026311,6.476608e+11,0.105374,-0.775373
6,0.00,,0.013046,2.708504e+12,0.000000,-0.888620
7,0.00,"spmeitc,spmcaphous,spmsnap,spmincwelfr,spmlunch",0.018742,2.600934e+12,0.001154,-0.839995
